# Beispielhafte Aufbereitung von Betroffenheitsvariablen

Die im Notebook verwendeten Dateien wurden am 18.12.2021 abgezogen.  
  
Aufgrund der ständig sich ändernden Lage können sich die Satzbeschreibungen im Laufe der Zeit verändert haben.  
  
Allgemeine Informationen zur Datenlage und zu den hier verwendeten Datenquellen sind auf https://www.corona-in-zahlen.de/datenquellen/ zusammengefasst dargestellt.

## Inhaltsverzeichnis  


[0. Vorarbeiten](#ch00)  

[1. Covid-19-Fallzahlen](#ch01)
>[1.1 Einlesen der Zeitreihe zu den Fallzahlen](#ch011)  
>[1.2 Aufbereitung der Fallzahlen - Berechnung von Summen](#ch012)  
>[1.3 Einlesen der aktuellen Fallzahlen](#ch013)  

  
[2. Zahlen zum Impfstatus](#ch02)
>[2.1 Einlesen der Daten zum Impfstatus](#ch021)  
>[2.2 Sonderschleife Berlin](#ch022)  
>[2.3 Zusammenspiel der Impfzahlen mit Basis-Dataframe](#ch023)  
  
[3. Berechnung kumulierter Fallzahlen und diverser Kennzahlen](#ch03) 
>[3.1 Abgeleitete Kennzahlen](#ch031)  
>[3.2 Aufbau Dataframe für Bundesland-Ebene und Gesamt-Deutschland](#ch032)  
  
[4. Mobilitätszahlen](#ch04)   
>[4.1 Landkreise](#ch041)  
>[4.2 Bundesländer](#ch042)  
>[4.3 Gesamt-Deutschland](#ch043)  
  
[5. DIVI-Zahlen](#ch05)  
>[5.1 Landkreise](#ch051)  
>[5.2 Bundesländer](#ch052)  
>[5.3 Gesamt-Deutschland](#ch053)  
  
[6. Hospitalisierung](#ch06) 
  
[7. Ausgabe](#ch07) 

<a id="ch00"></a>
## 0. Vorarbeiten

In [1]:
# verwendete Pakete
from platform import python_version
print('python ',python_version())

import locale
locale.setlocale(locale.LC_ALL,"")

import datetime as dt
from datetime import datetime, date, timedelta

import numpy as np
print('numpy ',np.version.full_version)
import pandas as pd
print('pandas ', pd.__version__)

import zipfile

python  3.8.8
numpy  1.20.1
pandas  1.2.4


In [2]:
# globale Variablen

# Einlese-/Ausgabe-Pfad
mypath = "C:/Users/3PKuehne/Daten/DAV/AGTarifierung/betroffenheit/daten/"

# Quellangaben zu den Dateien stehen in den jeweiligen Kapiteln,
# Dateien wurden jeweils heruntergeladen
in_rki_covid19_zr  = "rki_covid19.csv" # Zeitreihe zu den Fallzahlen nach Landkreis/Altersgruppe/Geschlecht
in_rki_covid19_akt = "RKI_Corona_Landkreise.csv"  # aktuelle Fallzahlen nah Landkreis
in_rki_impfzahlen  = "Aktuell_Deutschland_Landkreise_COVID-19-Impfungen.csv" # Zeitreihe Impfzahlen nach Landkreis/Alter/Impfung
in_mobilitaet_lkr  = "mobility_change_kreise.csv" # Zeitreihe Mobilitätsänderung nach Landkreis
in_mobilitaet_bl   = "mobility_change_bundeslaender.csv" # Zeitreihe Mobilitätsänderung nach Bundesland
in_mobilitaet_ges  = "mobility_change_germany.csv" # Zeitreihe Mobilitätsänderung nach Gesamt-Deutschland
in_schl_nuts       = "pc2020_DE_NUTS-2021_v3.0.zip" # Definition NUTS
in_schl_plz_kgs    = "zuordnung_plz_ort_landkreis.csv" # Definition amtlicher Gemeindeschlüssel
in_divi            = "zeitreihe-tagesdaten.csv" # Zeitreihe DIVI-Register nach Landkreis
in_hospital        = "Aktuell_Deutschland_COVID-19-Hospitalisierungen.csv" # Zeitreihe Hospitalisierung nach Bundesland

out_landkreis      = "tab_betroffenheit_landkreis.csv"
out_bundesland     = "tab_betroffenheit_bundesland.csv"
out_gesamt         = "tab_betroffenheit_deutschland.csv"

<a id="ch01"></a>
## 1. Covid-19-Fallzahlen
<a id="ch011"></a>
### 1.1 Einlesen der Zeitreihe zu den Fallzahlen

Quelle: `rki_covid.csv` von https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0/explore 
  
(Satzbeschreibung: https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_COVID19/FeatureServer/0)  

In [3]:
# Historie Inzidenzen
df_inzidenz_hist = pd.read_csv(mypath+in_rki_covid19_zr,encoding='utf-8')

# alle Feldnamen werden klein geschrieben
df_inzidenz_hist.rename(str.lower, axis='columns',inplace=True)

print(df_inzidenz_hist.shape)

df_inzidenz_hist.head().T

(3255422, 18)


,0,1,2,3,4
idbundesland,1,1,1,1,1
bundesland,Schleswig-Holstein,Schleswig-Holstein,Schleswig-Holstein,Schleswig-Holstein,Schleswig-Holstein
landkreis,SK Flensburg,SK Flensburg,SK Flensburg,SK Flensburg,SK Flensburg
altersgruppe,A35-A59,A15-A34,A35-A59,A15-A34,A05-A14
geschlecht,M,W,M,W,W
anzahlfall,1,1,1,1,2
anzahltodesfall,0,0,0,0,0
objectid,1,2,3,4,5
meldedatum,2021/03/25 00:00:00+00,2021/04/20 00:00:00+00,2021/03/26 00:00:00+00,2021/04/20 00:00:00+00,2021/10/17 00:00:00+00
idlandkreis,1001,1001,1001,1001,1001


In [4]:
df_inzidenz_hist.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3255422 entries, 0 to 3255421
Data columns (total 18 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   idbundesland          3255422 non-null  int64 
 1   bundesland            3255422 non-null  object
 2   landkreis             3255422 non-null  object
 3   altersgruppe          3255422 non-null  object
 4   geschlecht            3255422 non-null  object
 5   anzahlfall            3255422 non-null  int64 
 6   anzahltodesfall       3255422 non-null  int64 
 7   objectid              3255422 non-null  int64 
 8   meldedatum            3255422 non-null  object
 9   idlandkreis           3255422 non-null  int64 
 10  datenstand            3255422 non-null  object
 11  neuerfall             3255422 non-null  int64 
 12  neuertodesfall        3255422 non-null  int64 
 13  refdatum              3255422 non-null  object
 14  neugenesen            3255422 non-null  int64 
 15

Die beiden Felder `Meldedatum` und `RefDatum` liegen durch den Import as Text vor. Für die weitergehende Betrachtung interessiert nur die Datumsangabe, daher wird die Felder in 'klassische' Datumsfelder umgewandelt.

In [5]:
df_inzidenz_hist['meldedatum']=pd.to_datetime(df_inzidenz_hist['meldedatum'].str[:10])
df_inzidenz_hist['refdatum']=pd.to_datetime(df_inzidenz_hist['refdatum'].str[:10])
df_inzidenz_hist.head()

,idbundesland,bundesland,landkreis,altersgruppe,geschlecht,anzahlfall,anzahltodesfall,objectid,meldedatum,idlandkreis,datenstand,neuerfall,neuertodesfall,refdatum,neugenesen,anzahlgenesen,isterkrankungsbeginn,altersgruppe2
0,1,Schleswig-Holstein,SK Flensburg,A35-A59,M,1,0,1,2021-03-25,1001,"18.12.2021, 00:00 Uhr",0,-9,2021-03-24,0,1,1,Nicht übermittelt
1,1,Schleswig-Holstein,SK Flensburg,A15-A34,W,1,0,2,2021-04-20,1001,"18.12.2021, 00:00 Uhr",0,-9,2021-04-16,0,1,1,Nicht übermittelt
2,1,Schleswig-Holstein,SK Flensburg,A35-A59,M,1,0,3,2021-03-26,1001,"18.12.2021, 00:00 Uhr",0,-9,2021-03-19,0,1,1,Nicht übermittelt
3,1,Schleswig-Holstein,SK Flensburg,A15-A34,W,1,0,4,2021-04-20,1001,"18.12.2021, 00:00 Uhr",0,-9,2021-04-18,0,1,1,Nicht übermittelt
4,1,Schleswig-Holstein,SK Flensburg,A05-A14,W,2,0,5,2021-10-17,1001,"18.12.2021, 00:00 Uhr",0,-9,2021-10-17,0,2,0,Nicht übermittelt


<a id="ch012"></a>
### 1.2 Aufbereitung der Fallzahlen - Berechnung von Summen

Die Daten werden für die einzelnen Landkreise (Datenfeld idlandkreis) und die einzelnen Tage zusammengefasst. Die zusätzlich vorliegende Differenzierung nach Altersgruppe und Geschlecht wird hier ignoriert.

Vorab erfolgt eine Prüfung auf Vollständigkeit, da vereinzelt die Datei nicht vollständig vorlag.

Es gibt insgesamt 401 unterschiedliche Landkreis-ID. Allerdings weist das RKI für Berlin die Zahlen anch den Stadtbezirken (12 Stadtbezirke) aus, damit erhöht sich die Anzahl auf 411.

In [6]:
min_dat = min(df_inzidenz_hist.meldedatum)
max_dat = max(df_inzidenz_hist.meldedatum)
diff_dat = max_dat - min_dat

print('Die Zeitreihe läuft im Datensatz vom ',min_dat.strftime("%d.%m.%Y"),' bis zum ',max_dat.strftime("%d.%m.%Y"),'. ')
print('Damit sollten ',diff_dat.days+1,' unterschiedliche Meldedatum-Werte vorliegen.')

Die Zeitreihe läuft im Datensatz vom  02.01.2020  bis zum  17.12.2021 . 
Damit sollten  716  unterschiedliche Meldedatum-Werte vorliegen.


In [7]:
print("Das Feld idlandkreis hat "+str(df_inzidenz_hist['idlandkreis'].nunique())+" Ausprägungen.")
print("Das Feld meldedatum hat "+str(df_inzidenz_hist['meldedatum'].nunique())+" Ausprägungen.")

Das Feld idlandkreis hat 411 Ausprägungen.
Das Feld meldedatum hat 678 Ausprägungen.


Offensichtlich fehlen Meldedatum-Werte. Bei einem Abgleich mit dem RefDatum war die Differenz 1. 

Vergleiche mit den Veröffentlichungen und auch die Erläuterungen zum Covid-19-Dashboard zeigen, dass Die Berechnung von Inzidenzen auf Basis des Meldedatums erfolgen. 
  
Unabhängig davon wird für die Berechnung der abgeleiteten Kennzahlen ein DataFrame initialisiert, dass für jeden Landkreis jedes mögliche Meldedatum enthält.

In [8]:
# Meldedatum
base = min(df_inzidenz_hist.refdatum)
date_list = [base + dt.timedelta(days=x) for x in range(diff_dat.days+2)]
df_kgs = pd.DataFrame(date_list,columns = ['meldedatum'])

# Landkreis-ID
df2 = pd.DataFrame(df_inzidenz_hist['idlandkreis'].unique(),columns = ['kgs'])

df_kgs = pd.merge(df_kgs,df2, how='cross')

print(df_kgs.shape)

df_kgs.head()

(294687, 2)


,meldedatum,kgs
0,2020-01-01,1001
1,2020-01-01,1002
2,2020-01-01,1003
3,2020-01-01,1004
4,2020-01-01,1051


Im nächsten Schritt werden je Landkreis/Meldedatum die Fälle (Neuinfektion, Verstorben) zusammengefasst und an die Basisdatei angespielt.

In [9]:
df_test=df_inzidenz_hist.groupby(['idlandkreis','meldedatum'])[['anzahlfall','anzahltodesfall']].sum().reset_index()
df_test.head()

,idlandkreis,meldedatum,anzahlfall,anzahltodesfall
0,1001,2020-03-14,4,0
1,1001,2020-03-18,2,0
2,1001,2020-03-19,4,0
3,1001,2020-03-20,2,0
4,1001,2020-03-21,1,0


In [10]:
df_kgs = pd.merge(df_kgs,df_test, how='left', left_on=['meldedatum','kgs'],right_on=['meldedatum','idlandkreis'],copy=False
             ).drop(columns=['idlandkreis'],axis=1)

df_kgs.fillna(value={'anzahlfall': 0, 'anzahltodesfall': 0},inplace=True)
df_kgs.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 294687 entries, 0 to 294686
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   meldedatum       294687 non-null  datetime64[ns]
 1   kgs              294687 non-null  int64         
 2   anzahlfall       294687 non-null  float64       
 3   anzahltodesfall  294687 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 11.2 MB


In [11]:
# Löschen Dataframe df_inzidenz_hist
del(df_inzidenz_hist)

<a id="ch013"></a>
### 1.3 Einlesen der aktuellen Fallzahlen

Die Datei mit den aktuellen Fallzahlen enthält die Einwohnerzahlen, die für die Berechnung der Inzidenzen benötigt wird. Zusätzlich wird aus dieser Datei die Landkreisbezeichnung übernommen.  

Quelle: `RKI_Corona_Landkreise.csv` von https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/917fc37a709542548cc3be077a786c17_0/explore?location=51.159939%2C10.714458%2C6.71&showTable=true 
  
(Satzbeschreibung: https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_Landkreisdaten/FeatureServer/0)  

In [12]:
# aktuelle Inzidenzen
df_inzidenz_akt = pd.read_csv(mypath+in_rki_covid19_akt,encoding='utf-8')

# alle Feldnamen werden klein geschrieben
df_inzidenz_akt.rename(str.lower, axis='columns',inplace=True)

print(df_inzidenz_akt.shape)

pd.set_option('display.max_rows', None) 
df_inzidenz_akt.head().T

(411, 47)


,0,1,2,3,4
objectid,1,2,3,4,5
ade,4.0,4.0,4.0,4.0,4.0
gf,4.0,4.0,4.0,4.0,4.0
bsg,1.0,1.0,1.0,1.0,1.0
rs,1001,1002,1003,1004,1051
ags,1001.0,1002.0,1003.0,1004.0,1051.0
sdv_rs,10010000000.0,10020000000.0,10030000000.0,10040000000.0,10510044044.0
gen,Flensburg,Kiel,Lübeck,Neumünster,Dithmarschen
bez,Kreisfreie Stadt,Kreisfreie Stadt,Kreisfreie Stadt,Kreisfreie Stadt,Kreis
ibz,40.0,40.0,40.0,40.0,42.0


In [13]:
pd.reset_option('display.max_rows')

df_kgs = df_kgs.merge(df_inzidenz_akt[['rs','ewz','gen','bez']], how='left', left_on='kgs', right_on='rs'
                     ).drop(columns=['rs'],axis=1).rename(columns={"ewz": "einwohner", 'gen': 'kgs_name', 'bez': 'kgs_art'})

df_kgs.head()

,meldedatum,kgs,anzahlfall,anzahltodesfall,einwohner,kgs_name,kgs_art
0,2020-01-01,1001,0.0,0.0,89934,Flensburg,Kreisfreie Stadt
1,2020-01-01,1002,0.0,0.0,246601,Kiel,Kreisfreie Stadt
2,2020-01-01,1003,0.0,0.0,215846,Lübeck,Kreisfreie Stadt
3,2020-01-01,1004,0.0,0.0,79905,Neumünster,Kreisfreie Stadt
4,2020-01-01,1051,0.0,0.0,133251,Dithmarschen,Kreis


<a id="ch02"></a>
## 2. Zahlen zum Impfstatus
<a id="ch021"></a>
### 2.1 Einlesen der Daten zum Impfstatus
  
Quelle: https://github.com/robert-koch-institut/COVID-19-Impfungen_in_Deutschland/blob/master/Aktuell_Deutschland_Landkreise_COVID-19-Impfungen.csv 
  
(weitere Hinweise: https://github.com/robert-koch-institut/COVID-19-Impfungen_in_Deutschland)  

In [14]:
# Impfquoten
df_impfquoten =pd.read_csv(mypath+in_rki_impfzahlen,encoding='utf-8')

# alle Feldnamen werden klein geschrieben
df_impfquoten.rename(str.lower, axis='columns',inplace=True)

# Impfdatum in Datumsformat umformen
df_impfquoten['impfdatum']=pd.to_datetime(df_impfquoten['impfdatum'])

print(df_impfquoten.shape)

df_impfquoten.head()

(651398, 5)


C:\Users\3PKuehne\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,impfdatum,landkreisid_impfort,altersgruppe,impfschutz,anzahl
0,2020-12-27,1003,18-59,1,9
1,2020-12-27,1003,60+,1,19
2,2020-12-27,1053,18-59,1,13
3,2020-12-27,1053,60+,1,200
4,2020-12-27,1055,18-59,1,20


Die Landkreis-ID zum Impfort wird aus Vereinfachungsgründen mit der Landkreis-ID zu den Fallzahlen beim Mapping gleichgesetzt. Diese Unschärfe (Impfungen außerhalb des Landkreises vom eigenen Wohnort) liegt auch in den offiziellen Statistiken vor.  
  
Zusätzlich liegt das Problem vor, dass das Feld `landkreisid_impfort` auch die Ausprägung **_u_** für unbekannt kennt, wodurch das Format in der Spalte eine Mischung aus Zahl und Text ist. Die Zeilen mit dieser Ausprägung werden gelöscht und im Anschluss das Format in *Integer* umgewandelt.


In [15]:
df_impfquoten = df_impfquoten.loc[df_impfquoten.landkreisid_impfort != 'u'].copy()

df_impfquoten.shape

(650967, 5)

In [16]:
df_impfquoten['landkreisid_impfort'] = df_impfquoten['landkreisid_impfort'].astype('int')

In [17]:
min_dat = min(df_impfquoten.impfdatum)
max_dat = max(df_impfquoten.impfdatum)
diff_dat = max_dat - min_dat

print('Die Zeitreihe läuft im Datensatz vom ',min_dat.strftime("%d.%m.%Y"),' bis zum ',max_dat.strftime("%d.%m.%Y"),'. ')
print('Damit sollten ',diff_dat.days+1,' unterschiedliche Impfdatum-Werte vorliegen.')
print()
print("Das Feld landkreisid_impfort hat "+str(df_impfquoten['landkreisid_impfort'].nunique())+" Ausprägungen.")
print("Das Feld impfdatum hat "+str(df_impfquoten['impfdatum'].nunique())+" Ausprägungen.")

Die Zeitreihe läuft im Datensatz vom  27.12.2020  bis zum  17.12.2021 . 
Damit sollten  356  unterschiedliche Impfdatum-Werte vorliegen.

Das Feld landkreisid_impfort hat 402 Ausprägungen.
Das Feld impfdatum hat 356 Ausprägungen.


Auch die Impfzahlen werden je Landkreis/Datum zusammengefasst. Dabei wird je möglicher Ausprägung der Spalte `impfschutz` eine eigene Spalte erzeugt.

In [18]:
impf_stat = df_impfquoten.impfschutz.unique()

liste_impf = []
liste_impfq = []
for i in impf_stat:
    df_impfquoten['anz_impfung_'+str(i)] = df_impfquoten['anzahl'].where(df_impfquoten['impfschutz']==i)
    df_impfquoten['anz_impfung_'+str(i)].fillna(0,inplace=True)
    liste_impf += ['anz_impfung_'+str(i)]
    liste_impfq += ['quote_impfung_'+str(i)]
    
df_impfquoten.head()

,impfdatum,landkreisid_impfort,altersgruppe,impfschutz,anzahl,anz_impfung_1,anz_impfung_2,anz_impfung_3
0,2020-12-27,1003,18-59,1,9,9.0,0.0,0.0
1,2020-12-27,1003,60+,1,19,19.0,0.0,0.0
2,2020-12-27,1053,18-59,1,13,13.0,0.0,0.0
3,2020-12-27,1053,60+,1,200,200.0,0.0,0.0
4,2020-12-27,1055,18-59,1,20,20.0,0.0,0.0


In [19]:
df_impfquoten=df_impfquoten.groupby(['landkreisid_impfort','impfdatum'])[liste_impf].sum().reset_index()
df_impfquoten.head()

,landkreisid_impfort,impfdatum,anz_impfung_1,anz_impfung_2,anz_impfung_3
0,1001,2020-12-29,23.0,0.0,0.0
1,1001,2020-12-30,36.0,0.0,0.0
2,1001,2021-01-04,134.0,0.0,0.0
3,1001,2021-01-05,171.0,0.0,0.0
4,1001,2021-01-06,109.0,0.0,0.0


<a id="ch022"></a>
### 2.2 Sonderschleife Berlin
Für Berlin (KGS = 11000) werden bei den Fallzahlen die Werte je Stadtbezirk veröffentlicht, bei den Impfzahlen dagegen nur für die gesamte Stadt:

In [20]:
#Impfzahlen
print(pd.DataFrame(df_impfquoten.loc[np.logical_and(df_impfquoten.landkreisid_impfort >= 11000,
                                                    df_impfquoten.landkreisid_impfort < 12000)].landkreisid_impfort.unique()))

# Corona-Fallzahlen
pd.DataFrame(df_kgs.loc[np.logical_and(df_kgs.kgs >= 11000,df_kgs.kgs < 12000)].kgs.unique()).T

       0
0  11000


,0,1,2,3,4,5,6,7,8,9,10,11
0,11001,11002,11003,11004,11005,11006,11007,11008,11009,11010,11011,11012


Aus diesem Grund werden die vorhandenen Impfquoten für die gesamte Stadt als identisch angenommen und die Impfzahlen entsprechend der jeweiligen Einwohnerzahlen auf die einzelnen Stadtbezirke aufgeteilt.  

Mit dieser Modifikation können im Anschluss die Impfzahlen an den Basis-Dataframe angespielt werden.

In [21]:
# Sonderschleife Berlin
## Einwohnerzahlen
df_test = df_inzidenz_akt[['rs','ewz']].loc[np.logical_and(df_inzidenz_akt.rs >= 11000,
                                                           df_inzidenz_akt.rs < 12000)].copy()

ew_ges = sum(df_test.ewz)

## Impfzahlen
df_test_1 = df_impfquoten.loc[np.logical_and(df_impfquoten.landkreisid_impfort >= 11000,
                                             df_impfquoten.landkreisid_impfort < 12000)].copy()

for c, d in zip(liste_impf, liste_impfq):
    df_test_1[d] = df_test_1[c] / ew_ges
    
## Zusammenspiel    
df_test_2 = pd.merge(df_test[['rs','ewz']],df_test_1[['impfdatum']+liste_impfq], how='cross'
                    ).rename(columns={"ewz": "einwohner", 'rs': 'landkreisid_impfort'})

for c, d in zip(liste_impf, liste_impfq):
    df_test_2[c] = round(df_test_2[d] * df_test_2['einwohner'],0)

## Differenz wird im Stadtbezirk mit der höchsten ID bereinigt
df_test_3 = df_test_2.groupby(['impfdatum'])[liste_impf].sum().reset_index()

for c in liste_impf:
    df_test_3[c+'_org'] = df_test_3[c]

df_test_3 = df_test_3.drop(columns=liste_impf,axis=1).merge(df_test_1[['impfdatum']+liste_impf],
                                                            how='left',left_on='impfdatum',right_on='impfdatum').copy()

liste_impf_korr=[]
for c in liste_impf:
    df_test_3[c+'_diff'] = df_test_3[c+'_org']-df_test_3[c]
    liste_impf_korr += [c+'_diff']

df_test_2 = df_test_2.merge(df_test_3[['impfdatum']+liste_impf_korr],how='left',left_on='impfdatum',right_on='impfdatum').copy()

max_id = max(df_test.rs)
for c,d in zip(liste_impf,liste_impf_korr):
    df_test_2[d] = df_test_2[d].where(df_test_2.landkreisid_impfort == max_id, 0)
    df_test_2[c] = df_test_2[c] + df_test_2[d]
    
print('Impfquoten Berlin:',df_test_2.shape)

## Anspiel an Dataframe zu den Impfquoten
print('Impfquoten vor Ergänzung:',df_impfquoten.shape)
df_impfquoten = df_impfquoten.append(df_test_2[['landkreisid_impfort','impfdatum']+liste_impf], ignore_index=True)
print('Impfquoten nach Ergänzung:',df_impfquoten.shape)

## Löschen Dataframe Hilfs-Dataframes
del(df_test_1, df_test_2, df_test_3)

Impfquoten Berlin: (4272, 12)
Impfquoten vor Ergänzung: (126248, 5)
Impfquoten nach Ergänzung: (130520, 5)


<a id="ch023"></a>
### 2.3 Zusammenspiel der Impfzahlen mit Basis-Dataframe

In [22]:
# Anspiel Impfzahlen an Basis-Dataframe
df_kgs = df_kgs.merge(df_impfquoten[['landkreisid_impfort','impfdatum']+liste_impf], how='left', 
                      left_on=['kgs','meldedatum'], 
                      right_on=['landkreisid_impfort','impfdatum']
                     ).drop(columns=['landkreisid_impfort','impfdatum'],axis=1)

# Löschen Dataframe df_impfquoten
del(df_impfquoten)

dict_fillna = dict({})
for c in liste_impf:
    dict_fillna.update({c: 0})
    
df_kgs.fillna(value=dict_fillna,inplace=True)

df_kgs.head()

,meldedatum,kgs,anzahlfall,anzahltodesfall,einwohner,kgs_name,kgs_art,anz_impfung_1,anz_impfung_2,anz_impfung_3
0,2020-01-01,1001,0.0,0.0,89934,Flensburg,Kreisfreie Stadt,0.0,0.0,0.0
1,2020-01-01,1002,0.0,0.0,246601,Kiel,Kreisfreie Stadt,0.0,0.0,0.0
2,2020-01-01,1003,0.0,0.0,215846,Lübeck,Kreisfreie Stadt,0.0,0.0,0.0
3,2020-01-01,1004,0.0,0.0,79905,Neumünster,Kreisfreie Stadt,0.0,0.0,0.0
4,2020-01-01,1051,0.0,0.0,133251,Dithmarschen,Kreis,0.0,0.0,0.0


<a id="ch03"></a>
## 3. Berechnung kumulierter Fallzahlen und diverser Kennzahlen

In [23]:
# kumulierte Zahlen
for c in ['anzahlfall','anzahltodesfall']+liste_impf:
    df_kgs[c+'_kum'] = df_kgs.groupby(['kgs'])[c].cumsum()

<a id="ch031"></a>
### 3.1 Abgeleitete Kennzahlen
Im Folgenden werden folgende Kennzahlen berechnet:
- 7-Tage-Inzidenz
- R-Wert 4-Tage
- R-Wert 7-Tage

Alle Kennzahlen hatten im Laufe der Pandemie Auswirkungen auf die Maßnahmen von Bund/Land/Landkreis. Der Einfluss dieser Kennzahlen änderte sich im Laufe der Zeit.  
  
**_7-Tage-Inzidenz:_**  
Bei der **_7-Tage-Inzidenz_** liegt das Meldedatum beim Gesundheitsamt zugrunde, also das Datum, an dem das lokale Gesundheitsamt Kenntnis über den Fall erlangt und ihn elektronisch erfasst hat. Die Kennzahl beinhaltet die Anzahl der neu gemeldeten Fälle der jeweils letzten 7 Tage je 100.000 Einwohner.  
  
**_R-Wert:_**  
Die Berechnung des R-Wertes folgt dem Paper https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Projekte_RKI/R-Beispielrechnung.xlsx?__blob=publicationFile.   
  
Damit gilt:
- R-Wert 4-Tage: Anzahl neuer Fälle Meldedatum + 3 Vortage / Anzahl neuer Fälle aus dem Zeitraum 4. - 7. Vortag
- R-Wert 7-Tage: Anzahl neuer Fälle aus Zeitraum Folgetag - 5.Vortag / Anzahl neuer Fälle aus Zeitraum 3. - 9. Vortag

In [24]:
def kennzahlen(df, geo_group):
## 7-Tage-Inzidenz_ Meldedatum - 6 bis Meldedatum
## R-Wert-4 Nenner = Meldedatum - 3 bis Meldedatum + 0, Zähler = Meldedatum - 7 bis Meldedatum - 4
## R-Wert-7 Nenner = Meldedatum - 5 bis Meldedatum + 1, Zähler = Meldedatum - 9 bis Meldedatum - 3

    list_anz_tage = [7, 4, 4, 7, 7]
    list_tag_start = [6, 3, 7, 5, 9]
    list_name = ['anz_fall_7','oben_4','unten_4','oben_7','unten_7']
    for anz_tage, abzug_beg, anz_name in zip(list_anz_tage,list_tag_start,list_name):
    
        for i in range(anz_tage):
            df1 = df[[geo_group,'meldedatum']].copy()
            df1['melde_dat'] = (df1.meldedatum + timedelta(days= i - abzug_beg))
            if i == 0:
                df2 = df1.copy()
            else:
                df2 = df2.append(df1)

        df2 = df2.merge(df[[geo_group,'meldedatum','anzahlfall']],how='left',
                        left_on=[geo_group,'melde_dat'],right_on=[geo_group,'meldedatum']
                       ).drop(columns=['meldedatum_y'],axis=1).fillna(value={'anzahlfall': 0})

        df2 = df2.groupby([geo_group,'meldedatum_x'])['anzahlfall'].sum().reset_index().rename(columns={'anzahlfall': anz_name})

        df = df.merge(df2,how='left',left_on=[geo_group,'meldedatum'],
                      right_on=[geo_group,'meldedatum_x']).drop(columns=['meldedatum_x'],axis=1)
  
    df['inzidenz'] = round(df.anz_fall_7 / df.einwohner * 100000,1)
    df['r_wert_4'] = round(df.oben_4 / df.unten_4,2)
    df['r_wert_7'] = round(df.oben_7 / df.unten_7,2)
    df = df.drop(columns=list_name,axis=1)
    
    df['infektionsrate'] = round(df.anzahlfall_kum / df.einwohner,4)
    df['letalitaetsrate'] = round(df.anzahltodesfall_kum / df.anzahlfall_kum,4)
    
    dict_fillna = dict({'inzidenz': 0, 'r_wert_4': 0, 'r_wert_7': 0, 'infektionsrate': 0, 'letalitaetsrate': 0})
    for c,d in zip(liste_impf, liste_impfq):
        df[d] = round(df[c+'_kum'] / df.einwohner,4)
        dict_fillna.update({d: 0})
    
    df.fillna(value=dict_fillna,inplace=True)

    return df

In [25]:
%time df_kgs = kennzahlen(df_kgs, 'kgs')

Wall time: 3.08 s


In [26]:
df_kgs.loc[(df_kgs.kgs==1001) & 
           (df_kgs.meldedatum.dt.day>=10)& 
           (df_kgs.meldedatum.dt.month==12) & 
           (df_kgs.meldedatum.dt.year==2021)].T

,291399,291810,292221,292632,293043,293454,293865,294276
meldedatum,2021-12-10 00:00:00,2021-12-11 00:00:00,2021-12-12 00:00:00,2021-12-13 00:00:00,2021-12-14 00:00:00,2021-12-15 00:00:00,2021-12-16 00:00:00,2021-12-17 00:00:00
kgs,1001,1001,1001,1001,1001,1001,1001,1001
anzahlfall,23.0,30.0,17.0,10.0,32.0,26.0,40.0,40.0
anzahltodesfall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
einwohner,89934,89934,89934,89934,89934,89934,89934,89934
kgs_name,Flensburg,Flensburg,Flensburg,Flensburg,Flensburg,Flensburg,Flensburg,Flensburg
kgs_art,Kreisfreie Stadt,Kreisfreie Stadt,Kreisfreie Stadt,Kreisfreie Stadt,Kreisfreie Stadt,Kreisfreie Stadt,Kreisfreie Stadt,Kreisfreie Stadt
anz_impfung_1,106.0,101.0,0.0,47.0,156.0,130.0,91.0,29.0
anz_impfung_2,101.0,71.0,0.0,84.0,126.0,169.0,180.0,35.0
anz_impfung_3,1237.0,1476.0,10.0,847.0,1099.0,1537.0,1184.0,558.0


<a id="ch032"></a>
### 3.2 Aufbau Dataframe für Bundesland-Ebene und Gesamt-Deutschland

In [27]:
def zusammen(df,teiler,keze):
    liste_feld_kat  = ['kgs','meldedatum']
    liste_feld_sum  = ['einwohner']
    liste_feld_csum = ['anzahlfall','anzahltodesfall','anz_impfung_1','anz_impfung_2','anz_impfung_3']

    df_test = df[liste_feld_kat+liste_feld_sum+liste_feld_csum].copy()
    df_test['kgs'] = df_test['kgs'] / teiler
    df_test['kgs'] = df_test['kgs'].astype(int)

    df_test = df_test.groupby(liste_feld_kat)[liste_feld_csum+liste_feld_sum].sum().reset_index()
    df_test.sort_values(by=liste_feld_kat,inplace=True)
    
    geo_param = 'kgs'
    
    if keze == 1:
        geo_param = 'bl'
        df_test = df_test.rename(columns={'kgs':'bl'})
        df_test_1 = df_inzidenz_akt.groupby(['bl_id','bl']
                                           )['bl'].count().to_frame(name='anzahl'
                                                                   ).reset_index().rename(columns={'bl':'bl_name'})
        df_test = df_test.merge(df_test_1,how='left',left_on=['bl'],right_on=['bl_id']).drop(columns=['bl_id','anzahl'],axis=1)

# kumulierte Zahlen
    for c in ['anzahlfall','anzahltodesfall']+liste_impf:
        df_test[c+'_kum'] = df_test.groupby(geo_param)[c].cumsum()
    
    df_test = kennzahlen(df_test, geo_param)

    return df_test

In [28]:
%time df_bl = zusammen(df_kgs,1000,1) # Bundesland = ersten zwei Zeichen des KGS

print(df_bl.shape)
df_bl.tail().T

Wall time: 240 ms
(11472, 22)


,11467,11468,11469,11470,11471
bl,16,16,16,16,16
meldedatum,2021-12-13 00:00:00,2021-12-14 00:00:00,2021-12-15 00:00:00,2021-12-16 00:00:00,2021-12-17 00:00:00
anzahlfall,2546.0,3592.0,3215.0,3029.0,2103.0
anzahltodesfall,7.0,8.0,4.0,0.0,3.0
anz_impfung_1,1722.0,2812.0,2995.0,4174.0,2582.0
anz_impfung_2,1236.0,2469.0,2742.0,3404.0,2065.0
anz_impfung_3,14547.0,24341.0,25317.0,27217.0,18396.0
einwohner,2120237,2120237,2120237,2120237,2120237
bl_name,Thüringen,Thüringen,Thüringen,Thüringen,Thüringen
anzahlfall_kum,255934.0,259526.0,262741.0,265770.0,267873.0


In [29]:
%time df_gesamt = zusammen(df_kgs,100000,0) 

print(df_gesamt.shape)
df_gesamt.tail().T

Wall time: 121 ms
(717, 21)


,712,713,714,715,716
kgs,0,0,0,0,0
meldedatum,2021-12-13 00:00:00,2021-12-14 00:00:00,2021-12-15 00:00:00,2021-12-16 00:00:00,2021-12-17 00:00:00
anzahlfall,31296.0,59801.0,51289.0,45555.0,29172.0
anzahltodesfall,61.0,59.0,25.0,12.0,17.0
anz_impfung_1,48910.0,67835.0,99078.0,92057.0,87630.0
anz_impfung_2,54160.0,85602.0,121673.0,117134.0,101518.0
anz_impfung_3,664428.0,929892.0,1346402.0,1090151.0,917185.0
einwohner,83155031,83155031,83155031,83155031,83155031
anzahlfall_kum,6577689.0,6637490.0,6688779.0,6734334.0,6763506.0
anzahltodesfall_kum,107936.0,107995.0,108020.0,108032.0,108049.0


In [30]:
liste_drop = ['anz_impfung_1','anz_impfung_2','anz_impfung_3',
              'anz_impfung_1_kum','anz_impfung_2_kum','anz_impfung_3_kum']

df_kgs = df_kgs.drop(columns=liste_drop,axis=1)
df_bl = df_bl.drop(columns=liste_drop,axis=1)
df_gesamt = df_gesamt.drop(columns=liste_drop,axis=1)

<a id="ch04"></a>
## 4. Mobilitätszahlen
<a id="ch041"></a>
### 4.1 Landkreise

Quelle: https://github.com/rocs-org/covid-mobility-data/blob/main/data/mobility_change/mobility_change_kreise.csv  
  
(weitere Hinweise: https://github.com/rocs-org/covid-mobility-data)

In [31]:
# Mobilität
df_mobilitaet = pd.read_csv(mypath+in_mobilitaet_lkr,encoding='utf-8')

# alle Feldnamen werden klein geschrieben
df_mobilitaet.rename(str.lower, axis='columns',inplace=True)

# enthaltenes Datum in Datumsformat umformen
df_mobilitaet['date']=pd.to_datetime(df_mobilitaet['date'])

df_mobilitaet.head()

,nuts3,name,date,week,mobility_change,mobility_change_7day_average,mobility_change_weekly
0,DE111,"Stuttgart, Stadtkreis",2020-02-01,5,0.0819,NaN,0.0682
1,DE111,"Stuttgart, Stadtkreis",2020-02-02,5,0.0490,NaN,0.0682
2,DE111,"Stuttgart, Stadtkreis",2020-02-03,6,0.0477,NaN,0.0747
3,DE111,"Stuttgart, Stadtkreis",2020-02-04,6,0.0576,0.0649,0.0747
4,DE111,"Stuttgart, Stadtkreis",2020-02-05,6,0.0703,0.0708,0.0747


In [32]:
min_dat = min(df_mobilitaet.date)
max_dat = max(df_mobilitaet.date)
diff_dat = max_dat - min_dat

print('Die Zeitreihe läuft im Datensatz vom ',min_dat.strftime("%d.%m.%Y"),' bis zum ',max_dat.strftime("%d.%m.%Y"),'. ')
print('Damit sollten ',diff_dat.days+1,' unterschiedliche Datumswerte vorliegen.')
print()
print("Das Feld nuts3 hat "+str(df_mobilitaet['nuts3'].nunique())+" Ausprägungen.")
print("Das Feld date hat "+str(df_mobilitaet['date'].nunique())+" Ausprägungen.")

Die Zeitreihe läuft im Datensatz vom  01.02.2020  bis zum  13.12.2021 . 
Damit sollten  682  unterschiedliche Datumswerte vorliegen.

Das Feld nuts3 hat 401 Ausprägungen.
Das Feld date hat 682 Ausprägungen.


Bei der Mobilität wird abweichend vom RKI die NUTS-Systematik der Europäischen Union verwendet.

*Wikipedia per 18.12.2021:  
"**NUTS** bezeichnet eine hierarchische Systematik zur eindeutigen Identifizierung und Klassifizierung der räumlichen Bezugseinheiten der amtlichen Statistik in den Mitgliedstaaten der Europäischen Union. 
Sie lehnt sich eng an die Verwaltungsgliederung der einzelnen Länder an. In der Regel entspricht eine NUTS-Ebene einer Verwaltungsebene oder einer räumlichen Aggregation von Verwaltungseinheiten. Eine vergleichbare Systematik gibt es auch in den EFTA- und CEC-Ländern.  
Diese Systematik wurde 1980 vom Europäischen Amt für Statistik in Luxemburg entwickelt, um regionale Raumeinheiten innerhalb Europas auch international statistisch vergleichen zu können. NUTS-Regionen sind die Grundlage für die quantitative Beurteilung von Regionen durch die EU. Im Rahmen der Regionalpolitik werden Fördermittel konkreten NUTS-Regionen (vor allem NUTS-3-Regionen) zugewiesen."*  
  
Um die Mobiltätszahlen an den Dataframe anspielen zu können, müssen die NUTS-Werte in dien amtlichen Gemeindeschlüssel (KGS) überführt werden.  
  
Eine offizielle Übersicht zur Zuordnung einzelner PLZ zum NUTS3 (`pc2020_DE_NUTS-2021_v2.0.zip`) ist unter dem Link https://gisco-services.ec.europa.eu/tercet/flat-files zu finden.  
  
Eine kostenfreie Zuordnung der PLZ zum KGS (`zuordnung_plz_ort_landkreis.csv`) ist auf der Seite https://www.suche-postleitzahl.org/downloads zu finden. Die Aktualität hier muss nicht immer gegeben sein.  
  
Aus den GDV-Rundschreiben zu Regionalisierung in Kraftfahrt ist bekannt, dass es PLZ gibt, die zu mehreren Landkreisen (bzw. Bundesländern) gehören. Damit muss man bei Verwendung beider Tabellen für das Mapping Zuordnungen bei den nichteindeutigen Postleitzahlen treffen.  
  
Nach der Satzbeschreibung zu den aktuellen Fallzahlen (`RKI_Corona_Landkreise.csv`) ist jedoch eine Zuordnung KGS - NUTS3 inklusive der notwendigen Festlegung schon vorhanden. Auf diese wird im Folgenden zugegriffen. 


In [33]:
df_kgs_nuts = df_inzidenz_akt.groupby(['rs','nuts']
                                     )['nuts'].count().to_frame(name='anzahl'
                                                               ).reset_index().rename(columns={"rs": "ags5"}
                                                                                     ).drop(columns=['anzahl'],axis=1)

print(df_kgs_nuts.shape)

df_kgs_nuts.head()

(399, 2)


,ags5,nuts
0,1001,DEF01
1,1002,DEF02
2,1003,DEF03
3,1004,DEF04
4,1051,DEF05


In [34]:
# Prüfung auf Eindeutigkeit
df_test = df_kgs_nuts.groupby(['ags5'])['ags5'].count().to_frame(name='anzahl').reset_index()
df_test.loc[df_test.anzahl > 1]['anzahl'].count()

0

In den Fallzahlen wird Berlin nicht als Kreis (amtlicher 5-stelliger Schlüssel 11000 beinhaltet die gesamte Stadt), sondern für die einzelnen Stadtbezirke betrachtet. Aus diesem Grund wird hier explizit geprüft, welche Daten hier vorliegen und bei Bedarf notwendige Anpassungen für ein erfolgreiches Anspiel vorgenommen.

In [35]:
# Einträge zu Berlin im Mapping-Dataframe
df_kgs_nuts.loc[np.logical_and(df_kgs_nuts.ags5 >= 11000,df_kgs_nuts.ags5 < 12000)]

,ags5,nuts


In [36]:
# Einträge zu Berlin im Dataframe zu den aktuellen Fallzahlen
df_berlin = df_inzidenz_akt[['rs','nuts']].loc[np.logical_and(df_inzidenz_akt.rs >= 11000,df_inzidenz_akt.rs < 12000)]
df_berlin.T

,399,400,401,402,403,404,405,406,407,408,409,410
rs,11012,11004,11009,11003,11008,11011,11010,11005,11006,11001,11002,11007
nuts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Aufgrund der fehlenden Einträge zum *NUTS* in der DAtei mit den aktuellen Fallzahlen, fehlt der Eintrag zu Berlin im MApping-Dataframe komplett.  

Für Berlin wird daher auf die Zuordnungen PLZ-NUTS und PLZ-KGS zurückgegriffen.

In [37]:
# Hilfstabelle zum Umschlüsseln NUT3 in AGS (Kreisgemeindeschlüssel)
zf = zipfile.ZipFile(mypath+in_schl_nuts) 
df_nuts3_plz = pd.read_csv(zf.open('pc2020_DE_NUTS-2021_v3.0.csv'),delimiter=';',quotechar="'")

# alle Feldnamen werden klein geschrieben
df_nuts3_plz.rename(str.lower, axis='columns',inplace=True)

df_nuts3_plz.head()

,nuts3,code
0,DE111,70173
1,DE111,70174
2,DE111,70176
3,DE111,70178
4,DE111,70180


In [38]:
# Hilfstabelle zum Umschlüsseln PLZ in AGS (Kreisgemeindeschlüssel)
df_plz_ags = pd.read_csv(mypath+in_schl_plz_kgs)

# alle Feldnamen werden klein geschrieben
df_plz_ags.rename(str.lower, axis='columns',inplace=True)

df_plz_ags.loc[np.logical_and(df_plz_ags.ags >= 11000000,df_plz_ags.ags < 12000000)].head()

,osm_id,ags,ort,plz,landkreis,bundesland
949,62422,11000000,Berlin,10115,NaN,Berlin
950,62422,11000000,Berlin,10117,NaN,Berlin
951,62422,11000000,Berlin,10119,NaN,Berlin
952,62422,11000000,Berlin,10178,NaN,Berlin
953,62422,11000000,Berlin,10179,NaN,Berlin


In [39]:
# zu verwendender Kreisgemeindeschlüssel muss 5-stellig sein. Gleichzeitig Einschränkung auf Berlin
df_plz_ags['ags5'] = df_plz_ags.ags/1000
df_plz_ags['ags5'] = df_plz_ags['ags5'].astype(int)

df_plz_ags5 = df_plz_ags.loc[np.logical_and(df_plz_ags.ags5 >= 11000,
                                            df_plz_ags.ags5 < 12000)
                            ].groupby(['plz','ags5'])['ags5'].count().to_frame(name='anzahl').reset_index()
df_plz_ags5.head()

,plz,ags5,anzahl
0,10115,11000,1
1,10117,11000,1
2,10119,11000,1
3,10178,11000,1
4,10179,11000,1


In [40]:
# Mapping NUTS - KGS für Berlin
df_nuts3_ags = df_nuts3_plz.merge(df_plz_ags5[['plz','ags5']], how='inner', left_on='code', right_on='plz'
                                 ).groupby(['ags5','nuts3']
                                          )['ags5'].count().to_frame(name='anzahl'
                                                                    ).reset_index().rename(columns={"nuts3": "nuts"}
                                                                                          ).drop(columns=['anzahl'],axis=1)
df_nuts3_ags.head()

,ags5,nuts
0,11000,DE300


In [41]:
df_berlin['nuts'] = df_nuts3_ags.iloc[0]['nuts']
df_berlin = df_berlin.rename(columns={'rs':'ags5'})
df_berlin.T

,399,400,401,402,403,404,405,406,407,408,409,410
ags5,11012,11004,11009,11003,11008,11011,11010,11005,11006,11001,11002,11007
nuts,DE300,DE300,DE300,DE300,DE300,DE300,DE300,DE300,DE300,DE300,DE300,DE300


In [42]:
# Anfügen des Berlin-Mappings an den allgemeinen Mapping-Dataframe
print(df_kgs_nuts.shape)

df_kgs_nuts = df_kgs_nuts.append(df_nuts3_ags, ignore_index=True)
df_kgs_nuts = df_kgs_nuts.append(df_berlin)
print(df_kgs_nuts.shape)

(399, 2)
(412, 2)


Vor dem finalen Anspiel wird geprüft, ob wirklich für jeden NUTS-Wert eine KGS-Zuordnung erfolgreich war. Dazu wird für jeden NUTS, der in der offiziellen Datei vorliegt der KGS aus unserer Mapping-Datei angespielt.

In [43]:
df_test = pd.DataFrame(df_nuts3_plz['nuts3'].unique(),columns=['nuts3'])
df_test=df_test.merge(df_kgs_nuts[['nuts','ags5']], how = 'left',left_on = ['nuts3'],right_on=['nuts']
                     ).drop(columns=['nuts'],axis=1)

# Quelle: https://stackoverflow.com/questions/39421433/efficient-way-to-find-null-values-in-a-dataframe
null_counts = df_test.isnull().sum() # Zählen der Zeilen, in denen je Merkmal ein Wert fehlt
print(null_counts[null_counts > 0].sort_values(ascending=False)) # Auflistung in absteigender Form, sofern Anzahl > 0
print()

df_test.fillna(value={'ags5': 0},inplace=True)
df_test1 = df_test.loc[df_test.ags5==0].groupby(['nuts3','ags5'])['ags5'].count().to_frame(name='anzahl').reset_index()
nuts_fehl = df_test1['nuts3'].tolist()
print(nuts_fehl)
print()

print(df_kgs_nuts.shape)
df_kgs_nuts = df_kgs_nuts.loc[df_kgs_nuts.ags5!=0].copy()
print(df_kgs_nuts.shape)
df_test = df_nuts3_plz.loc[df_nuts3_plz.nuts3.isin(nuts_fehl)].merge(df_plz_ags[['plz','ags5']],how='left',
                                                                     left_on=['code'],right_on=['plz']
                                                                     ).rename(columns={'nuts3':'nuts'})
df_kgs_nuts = df_kgs_nuts.append(df_test[['ags5','nuts']],ignore_index=True)
print(df_kgs_nuts.shape)

ags5    1
dtype: int64

['DEG0N']

(412, 2)
(412, 2)
(413, 2)


In [44]:
# Ergänzung Mobilitätsübersicht um KGS
print(df_mobilitaet.shape)

df_mobilitaet = df_mobilitaet.merge(df_kgs_nuts[['ags5','nuts']], how='left', 
                                    left_on='nuts3', right_on='nuts'
                                   ).drop(['nuts'],axis=1)

df_mobilitaet['ags5'] = df_mobilitaet['ags5'].astype('int64')

print(df_mobilitaet.shape)

(273482, 7)
(281666, 8)


In [45]:
df_mobilitaet.loc[(df_mobilitaet.nuts3=='DE300')&
                  (df_mobilitaet.date.dt.day==1)& 
                  (df_mobilitaet.date.dt.month==2) & 
                  (df_mobilitaet.date.dt.year==2020)]

,nuts3,name,date,week,mobility_change,mobility_change_7day_average,mobility_change_weekly,ags5
95480,DE300,Berlin,2020-02-01,5,-0.0083,NaN,-0.0228,11000
95481,DE300,Berlin,2020-02-01,5,-0.0083,NaN,-0.0228,11012
95482,DE300,Berlin,2020-02-01,5,-0.0083,NaN,-0.0228,11004
95483,DE300,Berlin,2020-02-01,5,-0.0083,NaN,-0.0228,11009
95484,DE300,Berlin,2020-02-01,5,-0.0083,NaN,-0.0228,11003
95485,DE300,Berlin,2020-02-01,5,-0.0083,NaN,-0.0228,11008
95486,DE300,Berlin,2020-02-01,5,-0.0083,NaN,-0.0228,11011
95487,DE300,Berlin,2020-02-01,5,-0.0083,NaN,-0.0228,11010
95488,DE300,Berlin,2020-02-01,5,-0.0083,NaN,-0.0228,11005
95489,DE300,Berlin,2020-02-01,5,-0.0083,NaN,-0.0228,11006


Nach den Vorarbeiten werden alle drei Kennzahlen zur Änderung der Mobilität in den Basis-Dataframe übernommen.

In [46]:
# Löschen nicht mehr benötigter Zuordnungs-Dataframes
del(df_nuts3_plz, df_plz_ags, df_plz_ags5, df_nuts3_ags)

In [47]:
liste_mob = ['mobility_change', 'mobility_change_7day_average', 'mobility_change_weekly']
liste_drop = ['ags5','date']

df_kgs = df_kgs.merge(df_mobilitaet[liste_drop+liste_mob], how = 'left',
                      left_on=['kgs','meldedatum'],right_on=liste_drop
                     )
df_kgs = df_kgs.drop(columns=liste_drop,axis=1)

In [48]:
df_kgs.loc[(df_kgs.kgs==1001) & 
           (df_kgs.meldedatum.dt.day==10)& 
           (df_kgs.meldedatum.dt.month==12) & 
           (df_kgs.meldedatum.dt.year==2020)].T

,141384
meldedatum,2020-12-10 00:00:00
kgs,1001
anzahlfall,10.0
anzahltodesfall,0.0
einwohner,89934
kgs_name,Flensburg
kgs_art,Kreisfreie Stadt
anzahlfall_kum,356.0
anzahltodesfall_kum,4.0
inzidenz,30.0


<a id="ch042"></a>
### 4.2 Bundesländer

Quelle: https://github.com/rocs-org/covid-mobility-data/blob/main/data/mobility_change/mobility_change_bundeslaender.csv  
  
(weitere Hinweise: https://github.com/rocs-org/covid-mobility-data)

In [49]:
# Mobilität
df_mobilitaet = pd.read_csv(mypath+in_mobilitaet_bl,encoding='utf-8')

# alle Feldnamen werden klein geschrieben
df_mobilitaet.rename(str.lower, axis='columns',inplace=True)

# enthaltenes Datum in Datumsformat umformen
df_mobilitaet['date']=pd.to_datetime(df_mobilitaet['date'])

df_mobilitaet.head()

,nuts1,name,date,week,mobility_change,mobility_change_7day_average,mobility_change_weekly
0,DE1,Baden-Württemberg,2020-02-01,5,0.0821,NaN,0.0577
1,DE1,Baden-Württemberg,2020-02-02,5,0.0239,NaN,0.0577
2,DE1,Baden-Württemberg,2020-02-03,6,0.0709,NaN,0.0868
3,DE1,Baden-Württemberg,2020-02-04,6,0.0790,0.0765,0.0868
4,DE1,Baden-Württemberg,2020-02-05,6,0.0883,0.0788,0.0868


In [50]:
liste_mob = ['mobility_change', 'mobility_change_7day_average', 'mobility_change_weekly']
liste_drop = ['name','date']

df_bl = df_bl.merge(df_mobilitaet[liste_drop+liste_mob], how = 'left',
                      left_on=['bl_name','meldedatum'],right_on=liste_drop
                     )
df_bl = df_bl.drop(columns=liste_drop,axis=1)

In [51]:
df_bl.loc[(df_bl.bl==1) & 
          (df_bl.meldedatum.dt.day==10)& 
          (df_bl.meldedatum.dt.month==12) & 
          (df_bl.meldedatum.dt.year==2020)].T

,344
bl,1
meldedatum,2020-12-10 00:00:00
anzahlfall,460.0
anzahltodesfall,24.0
einwohner,2910875
bl_name,Schleswig-Holstein
anzahlfall_kum,17077.0
anzahltodesfall_kum,376.0
inzidenz,68.1
r_wert_4,1.48


<a id="ch043"></a>
### 4.3 Gesamt-Deutschland

Quelle: https://github.com/rocs-org/covid-mobility-data/blob/main/data/mobility_change/mobility_change_germany.csv  
  
(weitere Hinweise: https://github.com/rocs-org/covid-mobility-data)

In [52]:
# Mobilität
df_mobilitaet = pd.read_csv(mypath+in_mobilitaet_ges,encoding='utf-8')

# alle Feldnamen werden klein geschrieben
df_mobilitaet.rename(str.lower, axis='columns',inplace=True)

# enthaltenes Datum in Datumsformat umformen
df_mobilitaet['date']=pd.to_datetime(df_mobilitaet['date'])

df_mobilitaet.head()

,date,week,mobility_change,mobility_change_7day_average,mobility_change_weekly
0,2020-02-01,5,0.0324,0.0933,0.0111
1,2020-02-02,5,-0.0182,0.0764,0.0111
2,2020-02-03,6,0.0214,0.0742,0.0289
3,2020-02-04,6,0.0296,0.0546,0.0289
4,2020-02-05,6,0.0491,0.0489,0.0289


In [53]:
liste_mob = ['mobility_change', 'mobility_change_7day_average', 'mobility_change_weekly']
liste_drop = ['date']

df_gesamt = df_gesamt.merge(df_mobilitaet[liste_drop+liste_mob], how = 'left',
                            left_on=['meldedatum'],right_on=liste_drop
                           )
df_gesamt = df_gesamt.drop(columns=liste_drop,axis=1)

In [54]:
df_gesamt.tail().T

,712,713,714,715,716
kgs,0,0,0,0,0
meldedatum,2021-12-13 00:00:00,2021-12-14 00:00:00,2021-12-15 00:00:00,2021-12-16 00:00:00,2021-12-17 00:00:00
anzahlfall,31296.0,59801.0,51289.0,45555.0,29172.0
anzahltodesfall,61.0,59.0,25.0,12.0,17.0
einwohner,83155031,83155031,83155031,83155031,83155031
anzahlfall_kum,6577689.0,6637490.0,6688779.0,6734334.0,6763506.0
anzahltodesfall_kum,107936.0,107995.0,108020.0,108032.0,108049.0
inzidenz,392.6,381.2,362.1,347.6,321.3
r_wert_4,0.56,0.58,0.75,1.18,1.4
r_wert_7,0.9,0.87,0.86,0.82,0.73


In [55]:
del(df_mobilitaet)

<a id="ch05"></a>
## 5. DIVI-Zahlen
<a id="ch051"></a>
### 5.1 Landkreise


Quelle: Landkreis-Daten aus https://www.intensivregister.de/#/aktuelle-lage/downloads
  
(weitere Hinweise: https://edoc.rki.de/bitstream/handle/176904/7989/Tagesdaten%20CSV%20Erkl%c3%a4rung%20Stand%2029.3.pdf?sequence=1&isAllowed=y)

In [56]:
# DIVI
df_divi = pd.read_csv(mypath+in_divi)

# alle Feldnamen werden klein geschrieben
df_divi.rename(str.lower, axis='columns',inplace=True)

# enthaltenes Datum in Datumsformat umformen
df_divi['date']=pd.to_datetime(df_divi['date'])

df_divi.tail().T

,239608,239609,239610,239611,239612
date,2021-12-18 00:00:00,2021-12-18 00:00:00,2021-12-18 00:00:00,2021-12-18 00:00:00,2021-12-18 00:00:00
bundesland,16,16,16,16,16
gemeindeschluessel,16073,16074,16075,16076,16077
anzahl_standorte,1,1,1,1,1
anzahl_meldebereiche,1,1,1,1,2
faelle_covid_aktuell,12,4,2,3,4
faelle_covid_aktuell_invasiv_beatmet,11,3,1,1,4
betten_frei,4,2,1,0,7
betten_belegt,18,6,3,10,25
betten_belegt_nur_erwachsen,18,6,3,10,24


In [57]:
min_dat = min(df_divi.date)
max_dat = max(df_divi.date)
diff_dat = max_dat - min_dat

print('Die Zeitreihe läuft im Datensatz vom ',min_dat.strftime("%d.%m.%Y"),' bis zum ',max_dat.strftime("%d.%m.%Y"),'. ')
print('Damit sollten ',diff_dat.days+1,' unterschiedliche Datumswerte vorliegen.')
print()
print("Das Feld gemeindeschluessel hat "+str(df_divi['gemeindeschluessel'].nunique())+" Ausprägungen.")
print("Das Feld date hat "+str(df_divi['date'].nunique())+" Ausprägungen.")

Die Zeitreihe läuft im Datensatz vom  24.04.2020  bis zum  18.12.2021 . 
Damit sollten  604  unterschiedliche Datumswerte vorliegen.

Das Feld gemeindeschluessel hat 397 Ausprägungen.
Das Feld date hat 604 Ausprägungen.


In [58]:
df_divi['intensivbetten'] = round(df_divi.faelle_covid_aktuell / (df_divi.betten_frei + df_divi.betten_belegt),4)
df_divi.loc[(df_divi.gemeindeschluessel==3353) & 
            (df_divi.date.dt.day>=10) & 
            (df_divi.date.dt.month==12) & 
            (df_divi.date.dt.year==2021)].T

,236075,236472,236869,237266,237663,238060,238457,238854,239251
date,2021-12-10 00:00:00,2021-12-11 00:00:00,2021-12-12 00:00:00,2021-12-13 00:00:00,2021-12-14 00:00:00,2021-12-15 00:00:00,2021-12-16 00:00:00,2021-12-17 00:00:00,2021-12-18 00:00:00
bundesland,3,3,3,3,3,3,3,3,3
gemeindeschluessel,3353,3353,3353,3353,3353,3353,3353,3353,3353
anzahl_standorte,3,3,3,3,3,3,3,3,3
anzahl_meldebereiche,3,3,3,3,3,3,3,3,3
faelle_covid_aktuell,4,6,6,7,7,8,8,4,4
faelle_covid_aktuell_invasiv_beatmet,2,2,2,3,3,3,2,2,2
betten_frei,0,2,5,2,3,1,6,5,5
betten_belegt,32,30,27,30,29,31,26,27,27
betten_belegt_nur_erwachsen,32,30,27,30,29,31,26,27,27


In [59]:
# Zuspiel Landkreis
liste_mob = ['intensivbetten','faelle_covid_aktuell','faelle_covid_aktuell_invasiv_beatmet']
liste_drop = ['gemeindeschluessel','date']

df_kgs = df_kgs.merge(df_divi[liste_drop+liste_mob], how = 'left',
                      left_on=['kgs','meldedatum'],right_on=liste_drop,
                      
                     )
df_kgs = df_kgs.drop(columns=liste_drop,axis=1)

df_kgs.loc[(df_kgs.kgs==3353) & 
           (df_kgs.meldedatum.dt.day>=10) & 
           (df_kgs.meldedatum.dt.month==12) & 
           (df_kgs.meldedatum.dt.year==2021)].T

,291434,291845,292256,292667,293078,293489,293900,294311
meldedatum,2021-12-10 00:00:00,2021-12-11 00:00:00,2021-12-12 00:00:00,2021-12-13 00:00:00,2021-12-14 00:00:00,2021-12-15 00:00:00,2021-12-16 00:00:00,2021-12-17 00:00:00
kgs,3353,3353,3353,3353,3353,3353,3353,3353
anzahlfall,58.0,70.0,10.0,17.0,96.0,97.0,63.0,71.0
anzahltodesfall,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
einwohner,256016,256016,256016,256016,256016,256016,256016,256016
kgs_name,Harburg,Harburg,Harburg,Harburg,Harburg,Harburg,Harburg,Harburg
kgs_art,Landkreis,Landkreis,Landkreis,Landkreis,Landkreis,Landkreis,Landkreis,Landkreis
anzahlfall_kum,10804.0,10874.0,10884.0,10901.0,10997.0,11094.0,11157.0,11228.0
anzahltodesfall_kum,133.0,134.0,134.0,134.0,134.0,134.0,134.0,134.0
inzidenz,179.7,174.2,171.9,176.6,170.3,169.1,160.5,165.6


<a id="ch052"></a>
### 5.2 Bundesländer


In [60]:
# Zusammenfassung je Bundesland
liste_sum = ['faelle_covid_aktuell','betten_frei','betten_belegt','faelle_covid_aktuell_invasiv_beatmet']
df_divi_bl = df_divi.groupby(['date','bundesland'])[liste_sum].sum().reset_index()
df_divi_bl['intensivbetten'] = round(df_divi_bl.faelle_covid_aktuell / (df_divi_bl.betten_frei + df_divi_bl.betten_belegt),4)

df_divi_bl.loc[(df_divi_bl.bundesland==3) & 
               (df_divi_bl.date.dt.day>=10) & 
               (df_divi_bl.date.dt.month==12) & 
               (df_divi_bl.date.dt.year==2021)].T

,9522,9538,9554,9570,9586,9602,9618,9634,9650
date,2021-12-10 00:00:00,2021-12-11 00:00:00,2021-12-12 00:00:00,2021-12-13 00:00:00,2021-12-14 00:00:00,2021-12-15 00:00:00,2021-12-16 00:00:00,2021-12-17 00:00:00,2021-12-18 00:00:00
bundesland,3,3,3,3,3,3,3,3,3
faelle_covid_aktuell,247,251,256,247,250,249,249,249,245
betten_frei,327,312,310,330,320,310,307,319,334
betten_belegt,1749,1732,1724,1722,1775,1787,1790,1774,1730
faelle_covid_aktuell_invasiv_beatmet,138,139,147,150,142,144,145,143,138
intensivbetten,0.119,0.1228,0.1259,0.1204,0.1193,0.1187,0.1187,0.119,0.1187


In [61]:
# Zuspiel an df_bl
liste_mob = ['intensivbetten','faelle_covid_aktuell','faelle_covid_aktuell_invasiv_beatmet']
liste_drop = ['bundesland','date']

df_bl = df_bl.merge(df_divi_bl[liste_drop+liste_mob], 
                      how = 'left',left_on=['bl','meldedatum'],right_on=liste_drop
                     )
df_bl = df_bl.drop(columns=liste_drop,axis=1)

df_bl.loc[(df_bl.bl==3) & 
          (df_bl.meldedatum.dt.day>=10) & 
          (df_bl.meldedatum.dt.month==12) & 
          (df_bl.meldedatum.dt.year==2021)].T

,2143,2144,2145,2146,2147,2148,2149,2150
bl,3,3,3,3,3,3,3,3
meldedatum,2021-12-10 00:00:00,2021-12-11 00:00:00,2021-12-12 00:00:00,2021-12-13 00:00:00,2021-12-14 00:00:00,2021-12-15 00:00:00,2021-12-16 00:00:00,2021-12-17 00:00:00
anzahlfall,2502.0,1689.0,410.0,1546.0,2919.0,3351.0,2361.0,1632.0
anzahltodesfall,9.0,3.0,2.0,3.0,1.0,7.0,1.0,1.0
einwohner,8003421,8003421,8003421,8003421,8003421,8003421,8003421,8003421
bl_name,Niedersachsen,Niedersachsen,Niedersachsen,Niedersachsen,Niedersachsen,Niedersachsen,Niedersachsen,Niedersachsen
anzahlfall_kum,405712.0,407401.0,407811.0,409357.0,412276.0,415627.0,417988.0,419620.0
anzahltodesfall_kum,6659.0,6662.0,6664.0,6667.0,6668.0,6675.0,6676.0,6677.0
inzidenz,206.3,203.7,199.0,195.0,189.5,190.9,184.6,173.8
r_wert_4,1.61,1.3,0.81,0.54,0.55,0.8,1.36,1.67


In [62]:
# Intensivbettenbelegung auf Bundesland-Ebene wird zusätzlich an Landkreisübersicht gespielt, 
# da diese Kennzahl z.T. nur je Bundesland betrachtet wird

dict_liste = {'intensivbetten':'intensivbetten_bdl',
              'faelle_covid_aktuell':'faelle_covid_aktuell_bdl',
              'faelle_covid_aktuell_invasiv_beatmet':'faelle_covid_akt_invasiv_beatmet_bdl'}

df_kgs['bl'] = df_kgs.kgs/1000
df_kgs['bl'] = df_kgs.bl.astype('int')

df_kgs = df_kgs.merge(df_divi_bl[liste_drop+liste_mob].rename(columns=dict_liste), 
                      how = 'left',left_on=['bl','meldedatum'],right_on=liste_drop
                     )
df_kgs = df_kgs.drop(columns=liste_drop,axis=1)

df_kgs.loc[(df_kgs.kgs==3353) & 
           (df_kgs.meldedatum.dt.day>=10) & 
           (df_kgs.meldedatum.dt.month==12) & 
           (df_kgs.meldedatum.dt.year==2021)].T

,291434,291845,292256,292667,293078,293489,293900,294311
meldedatum,2021-12-10 00:00:00,2021-12-11 00:00:00,2021-12-12 00:00:00,2021-12-13 00:00:00,2021-12-14 00:00:00,2021-12-15 00:00:00,2021-12-16 00:00:00,2021-12-17 00:00:00
kgs,3353,3353,3353,3353,3353,3353,3353,3353
anzahlfall,58.0,70.0,10.0,17.0,96.0,97.0,63.0,71.0
anzahltodesfall,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
einwohner,256016,256016,256016,256016,256016,256016,256016,256016
kgs_name,Harburg,Harburg,Harburg,Harburg,Harburg,Harburg,Harburg,Harburg
kgs_art,Landkreis,Landkreis,Landkreis,Landkreis,Landkreis,Landkreis,Landkreis,Landkreis
anzahlfall_kum,10804.0,10874.0,10884.0,10901.0,10997.0,11094.0,11157.0,11228.0
anzahltodesfall_kum,133.0,134.0,134.0,134.0,134.0,134.0,134.0,134.0
inzidenz,179.7,174.2,171.9,176.6,170.3,169.1,160.5,165.6


<a id="ch053"></a>
### 5.3 Gesamt-Deutschland


In [63]:
liste_sum = ['faelle_covid_aktuell','betten_frei','betten_belegt','faelle_covid_aktuell_invasiv_beatmet']
liste_mob = ['intensivbetten','faelle_covid_aktuell','faelle_covid_aktuell_invasiv_beatmet']
liste_drop = ['date']

df_divi_bl = df_divi.groupby(['date'])[liste_sum].sum().reset_index()
df_divi_bl['intensivbetten'] = round(df_divi_bl.faelle_covid_aktuell / (df_divi_bl.betten_frei + df_divi_bl.betten_belegt),4)

df_gesamt = df_gesamt.merge(df_divi_bl[liste_drop+liste_mob], 
                            how = 'left',left_on=['meldedatum'],right_on=liste_drop
                           )
df_gesamt = df_gesamt.drop(columns=liste_drop,axis=1)

df_gesamt.loc[(df_gesamt.meldedatum.dt.day>=10) & 
              (df_gesamt.meldedatum.dt.month==12) & 
              (df_gesamt.meldedatum.dt.year==2021)].T

,709,710,711,712,713,714,715,716
kgs,0,0,0,0,0,0,0,0
meldedatum,2021-12-10 00:00:00,2021-12-11 00:00:00,2021-12-12 00:00:00,2021-12-13 00:00:00,2021-12-14 00:00:00,2021-12-15 00:00:00,2021-12-16 00:00:00,2021-12-17 00:00:00
anzahlfall,51008.0,34643.0,15461.0,31296.0,59801.0,51289.0,45555.0,29172.0
anzahltodesfall,101.0,50.0,18.0,61.0,59.0,25.0,12.0,17.0
einwohner,83155031,83155031,83155031,83155031,83155031,83155031,83155031,83155031
anzahlfall_kum,6496289.0,6530932.0,6546393.0,6577689.0,6637490.0,6688779.0,6734334.0,6763506.0
anzahltodesfall_kum,107807.0,107857.0,107875.0,107936.0,107995.0,108020.0,108032.0,108049.0
inzidenz,424.6,415.2,406.0,392.6,381.2,362.1,347.6,321.3
r_wert_4,1.44,1.19,0.79,0.56,0.58,0.75,1.18,1.4
r_wert_7,0.9,0.9,0.9,0.9,0.87,0.86,0.82,0.73


In [64]:
del(df_divi, df_divi_bl)

<a id="ch06"></a>
## 6. Hospitalisierung

Quelle: `Aktuell_Deutschland_COVID-19-Hospitalisierungen.csv`aus https://github.com/robert-koch-institut/COVID-19-Hospitalisierungen_in_Deutschland



In [65]:
# Hospitalisierung
df_hospital = pd.read_csv(mypath+in_hospital)

# alle Feldnamen werden klein geschrieben
df_hospital.rename(str.lower, axis='columns',inplace=True)

# enthaltenes Datum in Datumsformat umformen
df_hospital['datum']=pd.to_datetime(df_hospital['datum'])

df_hospital.tail().T

,78416,78417,78418,78419,78420
datum,2020-03-01 00:00:00,2020-03-01 00:00:00,2020-03-01 00:00:00,2020-03-01 00:00:00,2020-03-01 00:00:00
bundesland,Thüringen,Thüringen,Thüringen,Thüringen,Thüringen
bundesland_id,16,16,16,16,16
altersgruppe,05-14,15-34,35-59,60-79,80+
7t_hospitalisierung_faelle,0,0,0,0,0
7t_hospitalisierung_inzidenz,0.0,0.0,0.0,0.0,0.0


In [66]:
min_dat = min(df_hospital.datum)
max_dat = max(df_hospital.datum)
diff_dat = max_dat - min_dat

print('Die Zeitreihe läuft im Datensatz vom ',min_dat.strftime("%d.%m.%Y"),' bis zum ',max_dat.strftime("%d.%m.%Y"),'. ')
print('Damit sollten ',diff_dat.days+1,' unterschiedliche Datumswerte vorliegen.')
print()
print("Das Feld bundesland hat "+str(df_hospital['bundesland'].nunique())+" Ausprägungen.")
print("Das Feld datum hat "+str(df_hospital['datum'].nunique())+" Ausprägungen.")

Die Zeitreihe läuft im Datensatz vom  01.03.2020  bis zum  19.12.2021 . 
Damit sollten  659  unterschiedliche Datumswerte vorliegen.

Das Feld bundesland hat 17 Ausprägungen.
Das Feld datum hat 659 Ausprägungen.


Die Hospitalisierung liegt nicht auf Landkreis-Ebene vor. Die Zahlen je Bundesland werden daher an den Landkreis-Datensatz gespielt.  
  
Gleichzeitig fällt auf, dass das Feld bundesland 17 Ausprägungen enthält, obwohl es nur 16 Bundesländer gibt. Ursache hierfür ist das Vorliegen der Zahlen auch für Gesamt-Deutschland in diesem Datensatz.  

In [67]:
df_hospital.bundesland.unique()

array(['Bundesgebiet', 'Schleswig-Holstein', 'Hamburg', 'Niedersachsen',
       'Bremen', 'Nordrhein-Westfalen', 'Hessen', 'Rheinland-Pfalz',
       'Baden-Württemberg', 'Bayern', 'Saarland', 'Berlin', 'Brandenburg',
       'Mecklenburg-Vorpommern', 'Sachsen', 'Sachsen-Anhalt', 'Thüringen'],
      dtype=object)

Im Datensatz erfolgt eine Differenzierung nach Alter, die hier nicht weiter verfolgt wird.

In [68]:
df_hospital_m = df_hospital.groupby(['datum','bundesland_id'])[['7t_hospitalisierung_faelle']].sum().reset_index()

df_hospital_m.tail().T

,11198,11199,11200,11201,11202
datum,2021-12-19 00:00:00,2021-12-19 00:00:00,2021-12-19 00:00:00,2021-12-19 00:00:00,2021-12-19 00:00:00
bundesland_id,12,13,14,15,16
7t_hospitalisierung_faelle,318,294,610,467,652


In [69]:
# Zuspiel Einwohnerzahlen und Berechnung der Hospitalisierungs-Inzidenz

df_hospital_m = df_hospital_m.merge(df_bl[['meldedatum','bl','einwohner']].rename(columns={'einwohner':'ew_bl'}),
                                    how='left',left_on=['datum','bundesland_id'],right_on=['meldedatum','bl']
                                   )
df_hospital_m = df_hospital_m.drop(columns=['meldedatum','bl'],axis=1)

df_hospital_m = df_hospital_m.merge(df_gesamt[['meldedatum','kgs','einwohner']].rename(columns={'einwohner':'ew_ges'}),
                                    how='left',left_on=['datum','bundesland_id'],right_on=['meldedatum','kgs']
                                   )
df_hospital_m = df_hospital_m.drop(columns=['meldedatum','kgs'],axis=1)

df_hospital_m.fillna({'ew_bl':0, 'ew_ges':0},inplace=True) 

df_hospital_m['einwohner'] = df_hospital_m.ew_bl + df_hospital_m.ew_ges
df_hospital_m['hospitalisierung_inzidenz'] = round(df_hospital_m['7t_hospitalisierung_faelle'] * 100000 /
                                                   df_hospital_m.einwohner,4)

df_hospital_m.head().T

,0,1,2,3,4
datum,2020-03-01 00:00:00,2020-03-01 00:00:00,2020-03-01 00:00:00,2020-03-01 00:00:00,2020-03-01 00:00:00
bundesland_id,0,1,2,3,4
7t_hospitalisierung_faelle,48,2,0,0,0
ew_bl,0.0,2910875.0,1852478.0,8003421.0,680130.0
ew_ges,83155031.0,0.0,0.0,0.0,0.0
einwohner,83155031.0,2910875.0,1852478.0,8003421.0,680130.0
hospitalisierung_inzidenz,0.0577,0.0687,0.0,0.0,0.0


In [70]:
# Anspiel der Zahlen an Landkreis-Datensatz
liste_mob = ['hospitalisierung_inzidenz']
liste_drop = ['bundesland_id','datum']

df_kgs = df_kgs.merge(df_hospital_m[liste_drop+liste_mob].rename(columns=dict_liste), 
                      how = 'left',left_on=['bl','meldedatum'],right_on=liste_drop
                     )
df_kgs = df_kgs.drop(columns=liste_drop+['bl'],axis=1)

df_kgs.loc[(df_kgs.kgs==3353) & 
           (df_kgs.meldedatum.dt.day>=10) & 
           (df_kgs.meldedatum.dt.month==12) & 
           (df_kgs.meldedatum.dt.year==2021)].T

,291434,291845,292256,292667,293078,293489,293900,294311
meldedatum,2021-12-10 00:00:00,2021-12-11 00:00:00,2021-12-12 00:00:00,2021-12-13 00:00:00,2021-12-14 00:00:00,2021-12-15 00:00:00,2021-12-16 00:00:00,2021-12-17 00:00:00
kgs,3353,3353,3353,3353,3353,3353,3353,3353
anzahlfall,58.0,70.0,10.0,17.0,96.0,97.0,63.0,71.0
anzahltodesfall,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
einwohner,256016,256016,256016,256016,256016,256016,256016,256016
kgs_name,Harburg,Harburg,Harburg,Harburg,Harburg,Harburg,Harburg,Harburg
kgs_art,Landkreis,Landkreis,Landkreis,Landkreis,Landkreis,Landkreis,Landkreis,Landkreis
anzahlfall_kum,10804.0,10874.0,10884.0,10901.0,10997.0,11094.0,11157.0,11228.0
anzahltodesfall_kum,133.0,134.0,134.0,134.0,134.0,134.0,134.0,134.0
inzidenz,179.7,174.2,171.9,176.6,170.3,169.1,160.5,165.6


In [71]:
# Anspiel der Zahlen an Bundesland-Datensatz

df_bl = df_bl.merge(df_hospital_m[liste_drop+liste_mob].rename(columns=dict_liste), 
                      how = 'left',left_on=['bl','meldedatum'],right_on=liste_drop
                   )
df_bl = df_bl.drop(columns=liste_drop,axis=1)

df_bl.loc[(df_bl.bl==3) & 
          (df_bl.meldedatum.dt.day>=10) & 
          (df_bl.meldedatum.dt.month==12) & 
          (df_bl.meldedatum.dt.year==2021)].T

,2143,2144,2145,2146,2147,2148,2149,2150
bl,3,3,3,3,3,3,3,3
meldedatum,2021-12-10 00:00:00,2021-12-11 00:00:00,2021-12-12 00:00:00,2021-12-13 00:00:00,2021-12-14 00:00:00,2021-12-15 00:00:00,2021-12-16 00:00:00,2021-12-17 00:00:00
anzahlfall,2502.0,1689.0,410.0,1546.0,2919.0,3351.0,2361.0,1632.0
anzahltodesfall,9.0,3.0,2.0,3.0,1.0,7.0,1.0,1.0
einwohner,8003421,8003421,8003421,8003421,8003421,8003421,8003421,8003421
bl_name,Niedersachsen,Niedersachsen,Niedersachsen,Niedersachsen,Niedersachsen,Niedersachsen,Niedersachsen,Niedersachsen
anzahlfall_kum,405712.0,407401.0,407811.0,409357.0,412276.0,415627.0,417988.0,419620.0
anzahltodesfall_kum,6659.0,6662.0,6664.0,6667.0,6668.0,6675.0,6676.0,6677.0
inzidenz,206.3,203.7,199.0,195.0,189.5,190.9,184.6,173.8
r_wert_4,1.61,1.3,0.81,0.54,0.55,0.8,1.36,1.67


In [72]:
# Anspiel der Zahlen an Deutschlandland-Datensatz

df_gesamt = df_gesamt.merge(df_hospital_m[liste_drop+liste_mob], 
                            how = 'left',left_on=['kgs','meldedatum'],right_on=liste_drop
                           )
df_gesamt = df_gesamt.drop(columns=liste_drop+['kgs'],axis=1)

df_gesamt.loc[(df_gesamt.meldedatum.dt.day>=10) & 
              (df_gesamt.meldedatum.dt.month==12) & 
              (df_gesamt.meldedatum.dt.year==2021)].T

,709,710,711,712,713,714,715,716
meldedatum,2021-12-10 00:00:00,2021-12-11 00:00:00,2021-12-12 00:00:00,2021-12-13 00:00:00,2021-12-14 00:00:00,2021-12-15 00:00:00,2021-12-16 00:00:00,2021-12-17 00:00:00
anzahlfall,51008.0,34643.0,15461.0,31296.0,59801.0,51289.0,45555.0,29172.0
anzahltodesfall,101.0,50.0,18.0,61.0,59.0,25.0,12.0,17.0
einwohner,83155031,83155031,83155031,83155031,83155031,83155031,83155031,83155031
anzahlfall_kum,6496289.0,6530932.0,6546393.0,6577689.0,6637490.0,6688779.0,6734334.0,6763506.0
anzahltodesfall_kum,107807.0,107857.0,107875.0,107936.0,107995.0,108020.0,108032.0,108049.0
inzidenz,424.6,415.2,406.0,392.6,381.2,362.1,347.6,321.3
r_wert_4,1.44,1.19,0.79,0.56,0.58,0.75,1.18,1.4
r_wert_7,0.9,0.9,0.9,0.9,0.87,0.86,0.82,0.73
infektionsrate,0.0781,0.0785,0.0787,0.0791,0.0798,0.0804,0.081,0.0813


<a id="ch07"></a>
## 7. Ausgabe


In [73]:
df_kgs.to_csv(mypath+out_landkreis,encoding='utf-8',index=False,sep=';',decimal=',')
df_bl.to_csv(mypath+out_bundesland,encoding='utf-8',index=False,sep=';',decimal=',')
df_gesamt.to_csv(mypath+out_gesamt,encoding='utf-8',index=False,sep=';',decimal=',')